In [116]:
import pandas as pd

In [117]:
DATA = 'ech_dataset'

In [118]:
articles = !ls echr_dataset

In [119]:
for article in articles:
    path = f'{DATA}/{article}'

In [170]:
!ls echr_dataset/Article6/

cases_a6.csv		     ngrams_a6_full.csv       ngrams_a6_relevantLaw.csv
ngrams_a6_circumstances.csv  ngrams_a6_law.csv	      topics6.csv
ngrams_a6_featureNames.txt   ngrams_a6_procedure.csv  topics6_vocab.txt


In [243]:
def get_article(path, features):
    files_raw = !ls {path}
    files_features = ['cases', 'circumstances', 'featureNames', 'full', 'law', 'procedure', 'relevantLaw', 'topics', 'topicsVocab']
    files = dict(zip(files_features, files_raw))
    
    group = []
    for feature in features:
        if(feature == 'topics'): raw = pd.read_csv(path+files[feature], sep='\t', header=None)
        elif(feature == 'topicsVocab'): raw = open(path+files[feature]).read(); raw.replace('\n', '').replace(', ', ',').split(',')
        else: raw = pd.read_csv(path+files[feature], header=None)
        
        if feature == 'cases': raw = raw[1]
            
        group.append(raw)
    return pd.concat(group, axis=1, ignore_index=True)

In [254]:
!ls echr_dataset

Article3  Article6  Article8


In [263]:
data = get_article('echr_dataset/Article8/', ['cases', 'full'])

In [264]:
X = data.iloc[:, 1:]

In [265]:
y = data[0]

In [266]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

import numpy as np

In [267]:
param_grid = [
    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
]

In [268]:
clf = make_pipeline(preprocessing.StandardScaler(), 
                    GridSearchCV(SVC(),
                                 param_grid=param_grid, cv=10, refit=True))

In [269]:
scores = cross_val_score(clf, X, y, cv=10)

In [270]:
np.mean(scores)

0.7115384615384616